# Safe and Just Earth Systems Boundaries: Planetary Safe Water Boundaries and Environmental Flow Analysis

***
<font color=red>Title: Safe and Just Earth Systems Boundaries: Planetary Safe Water Boundaries and Environmental Flow Analysis  
Author: Pamela A. Green  
Date: February 14, 2023  

This code was developed between December 1, 2021 and February 14, 2023 by Pamela Green, Senior Research Associate, CUNY Advanced Science Research Center, New York, NY under subcontract to Griffith University.  </font>
***

This code represents spatial modelling for development of the safe and just surface water target for Working Group 3 of the Earth Commission for the Earth Commission Long Report and the <i><b>"Safe and Just Earth Systems Boundaries"</b></i> publication.

Specific elements of the surface water target are: 
1. Spatial modelling of the extent of global-scale hydrological alteration
2. Spatial analysis of population, hydrological alteration and groundwater decline for the analysis of surface just access to surface water.

Environmental stress is defined as an alteration of flow between pristine (non-human impacted) and disturbed (human impacted) conditions that exceed a given threshold. We derived the pristine and disturbed monthly river flow datasets from the WBM water balance model river discharge outputs (Wisser et al. 2010) at 6-minute grid cell resolution using the TerraClimate high resolution data set of monthly climate forcings (Abatzoglou, et al. 2018) for the period 2000-2020. River basin delineation and flow routing configurations are defined by the WBM 6-minute topological river network used to establish local discharge and river flow (Wisser et al. 2010). The pristine and disturbed WBM runs use the same climate forcings for the 2000-2020 time period but only employ human alterations to the water cycle, including water extraction for irrigation and large reservoirs, in the disturbed runs. The modelled long-term mean contemporary global annual discharge of 38,000km3 under this scenario is consistent with results from the literature (REFS, table from Nature synthesis??). 

Long-term mean monthly discharge is calculated for the modelled pristine (non-human impacted) and disturbed (human impacted) discharge from the WBM model over the 2000-2020 time domain to determine the extent of altered flow. The analysis is limited to only the perennial or actively flowing river extents by applying a 3mm/yr upstream monthly average runoff exceedance threshold (Fekete et al. 2001) occurring for at least 10 years out of the 2000-2020 time domain. We also mask out upstream headwater areas (smaller than 250km2) that have modelled irrigation depths below the median irrigation depth for small headwater cells (3.6 mm/yr). This mask is applied to eliminate noise in the modelled data associated with very low irrigation and discharge values in headwater grid cells.

To quantify the extent of alteration of surface flows, we first calculate the number of months in a year where the contemporary disturbed discharge is more than 20% different from pre-industrial discharge using the long-term mean gridded discharge data. We then represent this data as the proportion of months in a year with more than 20% difference (divide by 12 months/year). Finally, we calculate a basin-level estimate of the proportion of grid cells where discharge is more than 20% different by taking the basin-wide mean of the gridded proportion of months over the actively flowing river extents within the basin. We also calculate the population weighted average of this index using the GPw4 population data (CIESIN 2018) for 2020, and a discharge weighted average of the index using the long-term mean pre-industrial flows from our model runs. River network and basin extents are defined by the WBM water balance model with naming convention taken from the GRDC Major River Basins of the World (GRDC 2020).  

Safe water deficits are defined as the difference between water demand metrics and available surface water. Available surface water is defined as the amount of flow minus what is needed to support natural systems and is calculated as 20% of the long term mean annual pristine discharge volume. Safe water demands are defined by four metrics expressed as volume of demand per capita (Table below). Safe water demands are converted to spatially distributed gridded volumes by multiplying the demand metrics by a distributed population dataset for year 2020 (CIESIN 2018) and then summed over river basins. Long term mean annual discharge and available surface water discharge at basin mouth is used to define integrated water flows for the river basins.

| Water Demand Metric | Values (Liters/capita/day)
| --- | --- |
| Domestic Dignity | 50
| Domestic Capability | 100
| All Needs Dignity | 292.85
| All Needs Capability | 405.67

The Safe Water Deficit by river basin is calculated as the amount of water demand not met by the available surface water and is represented as a total volume as well as a proportion of the available surface water (Fig_DefecitProportion_AllCap.png - Figure C). Fig_DefecitProportion_AllCap-B also shows the Safe Water Deficit as a proportion of total pristine discharge over river basins.

Safe Water Deficit = Water DemandBasin – Available Surface WaterBasin
where Water DemandBasin > Available Surface WaterBasin

Available Surface WaterBasin = Long Term Mean Pristine DischargeBasin * 0.2


Where we can't meet safe water needs with surface water, we may be able to rely on groundwater resources to meet these needs. All Groundwater data was provided by Christopher Ndehedehe from Griffith University (c.ndehedehe@griffith.edu.au).Groundwater recharge volumes are summed over river basins for basin-level calculations. We calculate the proportion of groundwater needed to meet the safe water deficits as the ratio of the Safe Water Deficit and the Safely Available Groundwater from Recharge (Fig_DefecitProportion_AllCap.png - Figure A).

Proportion of Groundwater Needed to Meet Safe Water Deficit = 
Safe Water DeficitBasin / Safely Available Groundwater from Recharge Basin

Finally, safe water demand not met by either surface water or groundwater recharge is calculated (Columns AE-AH) as the amount of safe water deficit in excess of the safely available groundwater recharge:

Unmet Safe Water Deficit = Safe Water DeficitBasin – Safely Available Groundwater from Recharge Basin 
where Safe Water DeficitBasin > Safely Available Groundwater from Recharge Basin

<b>References:</b>

Abatzoglou, John T., Solomon Z. Dobrowski, Sean A. Parks, and Katherine C. Hegewisch. “TerraClimate, a High-Resolution Global Dataset of Monthly Climate and Climatic Water Balance from 1958–2015.” Scientific Data 5, no. 1 (January 9, 2018): 170191. https://doi.org/10.1038/sdata.2017.191.

Center for International Earth Science Information Network - CIESIN - Columbia University. 2018. Gridded Population of the World, Version 4 (GPWv4): Population Density, Revision 11. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/H49C6VHW.

Fekete, Balázs M., Charles J. Vörösmarty, and Richard B. Lammers. “Scaling Gridded River Networks for Macroscale Hydrology: Development, Analysis, and Control of Error.” Water Resources Research 37, no. 7 (July 2001): 1955–67. https://doi.org/10.1029/2001WR900024.

GRDC (2020): Major River Basins of the World / Global Runoff Data Centre, GRDC. 2nd, rev. ext. ed. Koblenz, Germany: Federal Institute of Hydrology (BfG).

Wisser, D., B. M. Fekete, C. J. Vörösmarty, and A. H. Schumann. “Reconstructing 20th Century Global Hydrography: A Contribution to the Global Terrestrial Network- Hydrology (GTN-H).” Hydrology and Earth System Sciences 14, no. 1 (January 6, 2010): 1–24. https://doi.org/10.5194/hess-14-1-2010.


In [ ]:
import numpy as np
import pandas as pd
import scipy

from scipy import stats
from osgeo import gdal, gdal_array, osr, ogr
from osgeo.gdalconst import *
import xarray as xr

from math import floor

# Modules and Functions

## Single band numpy array to GeoTiff file

In [ ]:
def save2File(rA, outname, nrows, ncols, geo_transform):  # Resolution for the 6min
    # create the output image
    # Note that in the geo transform the third and sixth parameters are equal to the Arc/Info generate fishnet
    # Y-coordinate paramenter (defining the rotation of the final grid)
    # LCORD
    outDs = gdal.GetDriverByName('GTiff').Create(outname, ncols, nrows, 1, gdal.GDT_Float32)
    outBand = outDs.GetRasterBand(1)
    outBand.WriteArray(rA)
    outDs.SetGeoTransform(geo_transform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    outDs.SetProjection(srs.ExportToWkt())
    outDs = None

    """
    geotransform[0] = top left x
    geotransform[1] = w-e pixel resolution
    geotransform[2] = 0
    geotransform[3] = top left y
    geotransform[4] = 0
    geotransform[5] = n-s pixel resolution (negative value)
    """

****
## MultiBand numpy array to GeoTiff

In [ ]:
def CreateMultiGeoTiff(Array, Name, driver, NDV, GeoT, Projection, DataType):
    Array[np.isnan(Array)] = NDV
    DataSet = gdal.GetDriverByName(driver).Create(Name, Array.shape[2], Array.shape[1], Array.shape[0], DataType)
    DataSet.SetGeoTransform(GeoT)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(Projection)
    DataSet.SetProjection(srs.ExportToWkt() )
    for i, image in enumerate(Array, 1):
        DataSet.GetRasterBand(i).WriteArray( image )
        DataSet.GetRasterBand(i).SetNoDataValue(NDV)
    DataSet.FlushCache()
    return Name

# Global Variables, Definitions, Datasets

## Global Pathways and Data files

In [ ]:
main = '/Users/pamelagreen/Desktop/Data/'

out_dir = str(main) + 'Griffith2022/'
TCdirMain = str(main) + 'TerraClimate/'
TCdir = str(main) + 'TerraClimate/wbm/'

cellarea = str(main) + 'cellarea/CellArea_6m.tif'
upAreafile= str(main) + 'cellarea/Global_UpstreamAreakm2_6min.tif'

## Explicitely define projection information for 6 minute rasters

In [ ]:
n = 300

ncols6m = 3600
nrows6m = 1500
xll6m = -180
yll6m = -60
xur6m = 180
yur6m = 90
cellsize6m = 0.1
cellang = cellsize6m * -1
nodata = -9999
geotr6m = ([ xll6m, cellsize6m, 0, yur6m, 0, cellang ])

geodriver = 'GTiff'
geoproj = 4326

head6m = 'ncols ' + str(ncols6m) + '\nnrows ' + str(nrows6m) + '\nxllcorner ' + str(xll6m) + '\nyllcorner ' + str(yll6m) + '\ncellsize ' + str(cellsize6m) + '\nNODATA_value ' + str(nodata)

## Read in GHAAS3 Basin and Subbasin, Cellarea, Uptsream Area, Ocean Boundary, and GADM Continent grids at 6min resolution

In [ ]:
fin = str(out_dir) + 'RiverBasin/HydroSTN06_Subbasin.tif'
ds = gdal.Open(str(fin))
subbas = np.array(ds.GetRasterBand(1).ReadAsArray())

del ds

fin = str(out_dir) + 'RiverBasin/Global_Basin_HydroSTN30_06min_GHAAS3.tif'
ds = gdal.Open(str(fin))
basin = np.array(ds.GetRasterBand(1).ReadAsArray())
basin[np.isnan(basin)] = 0

basin_name_file = 'MajorBasinLookUp.xlsx'
pd_basin_name = pd.read_excel(out_dir + 'RiverBasin/' + basin_name_file, sheet_name='LookupVALUES')
del pd_basin_name['BasinName']
del pd_basin_name['CONTINENT']
del pd_basin_name['SEA']
del pd_basin_name['OCEAN']
del pd_basin_name['TFDD']
pd_basin_name = pd_basin_name[(pd_basin_name['ID'] != 0)]

fin = str(upAreafile)
ds = gdal.Open(str(fin))
UpArea = np.array(ds.GetRasterBand(1).ReadAsArray())
UpArea[np.isnan(UpArea)] = 0

fin = str(cellarea)
ds = gdal.Open(str(fin))
CellArea = np.array(ds.GetRasterBand(1).ReadAsArray())
CellArea[np.isnan(CellArea)] = 0

sum_basin_area = scipy.ndimage.sum(CellArea, basin, basin)
sum_area = scipy.ndimage.sum(CellArea, subbas, subbas)

fin = str(TCdirMain) + '/ocean.tif'
ds = gdal.Open(str(fin))
ocean = np.array(ds.GetRasterBand(1).ReadAsArray())

## Establish actively flowing areas
Masks out nonflowing areas. Based on Upstream Runoff less than 3mm occuring 10 or more times in any given year over the time period of study (2000-2021). Problematic low- and non-flow basins removed due to rgis difficulties in dry regions.

In [ ]:
StartYear=2000
EndYear=2021

upro_dir = str(TCdir)

for x in range(StartYear, EndYear):
    fin = upro_dir + 'UpRO/UpRO' + str(x) + 'p.tif'
    ds = gdal.Open(str(fin))
    UpROp = np.array(ds.GetRasterBand(1).ReadAsArray())
    UpROp[np.isnan(UpROp)] = 0
    UpROp = UpROp[:, :]
    
    maskp = np.where(UpROp <= 3.0000, 1, 0)
    if x == StartYear:
        mask_yr = maskp
        maskp_yr = maskp
    else:
        mask_yr = maskp_yr + maskp
        maskp_yr = maskp_yr + maskp

flow_mask = np.where(mask_yr >= 10.0000, 1, 0)
flow_mask = np.resize(flow_mask, (1,nrows6m,ncols6m))

flow_mask = np.where(subbas == 21599, 1, flow_mask)
flow_mask = np.where(subbas == 21597, 1, flow_mask)
flow_mask = np.where(subbas == 21596, 1, flow_mask)
flow_mask = np.where(subbas == 21595, 1, flow_mask)
flow_mask = np.where(subbas == 21593, 1, flow_mask)
flow_mask = np.where(subbas == 21587, 1, flow_mask)
flow_mask = np.where(subbas == 21583, 1, flow_mask)
flow_mask = np.where(subbas == 21598, 1, flow_mask)

outname = out_dir + 'UpRO/UpROmask_year.tif'
save2File(mask_yr[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'UpRO/UpROmaskPRIST_year.tif'
save2File(maskp_yr[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'UpRO/flow_mask.tif'
save2File(flow_mask[0,:, :], outname, nrows6m, ncols6m, geotr6m)

######################################################
## Can simple read in from UpRO/flow_mask.tif after it has been created above
######################################################
# fin = out_dir + 'UpRO/flow_mask.tif'
# ds = gdal.Open(str(fin))
# flow_mask = np.array(ds.GetRasterBand(1).ReadAsArray())
# flow_mask[np.isnan(flow_mask)] = 0

# Import Surface Water, Groundwater and Population Datasets

## Read in Downstream Population, Local Pop scaling from 2020

Population datasets are from CIESIN GPW4(references below) and were resampled to 6min resolution.

Contemporary Population:
- Center for International Earth Science Information Network - CIESIN - Columbia University. 2018. Gridded Population of the World, Version 4 (GPWv4): Population Density, Revision 11. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC).


In [ ]:
fin = str(main) + 'population/gpw4/6min/gpw_v4_population_count_rev11_2020_6min.tif'
ds = gdal.Open(str(fin))
pop2020 = np.array(ds.GetRasterBand(1).ReadAsArray())
pop2020[np.isnan(pop2020)] = 0
pop2020 = pop2020[ :-n, :]

fin = str(main) + 'population/dwnpop/6min/dwnpop_2020_gpwv4_TerraClimate_aLTM2000-2020_6min.tif'
ds = gdal.Open(str(fin))
dwnpop2020 = np.array(ds.GetRasterBand(1).ReadAsArray())
dwnpop2020[np.isnan(dwnpop2020)] = 0

sum_pop2020 = scipy.ndimage.sum(pop2020, subbas, subbas)
sum_dwnpop2020 = scipy.ndimage.sum(dwnpop2020, subbas, subbas)

sum_basin_pop2020 = scipy.ndimage.sum(pop2020, basin, basin)
sum_basin_dwnpop2020 = scipy.ndimage.sum(dwnpop2020, basin, basin)

## Import Groundwater Risk, Groundwater Recharge and Trends; convert to km3/yr and sum by basin

All Groundwater data was provided by Christopher Ndehedehe from Griffith University (c.ndehedehe@griffith.edu.au)

In [ ]:
infn = str(out_dir) + 'Groundwater/RiskMap_6m.tif'

ds = gdal.Open(str(infn))
gw_mm = np.array(ds.GetRasterBand(1).ReadAsArray())
gw_mm[np.isnan(gw_mm)] = 0
gw_mm = np.where(gw_mm <= -9999, 0.00, gw_mm)

gw_km3 = gw_mm * CellArea / 1000000
sum_basin_gwkm3 = scipy.ndimage.sum(gw_km3, basin, basin)
sum_gwkm3 = scipy.ndimage.sum(gw_km3, subbas, subbas)

infn = str(out_dir) + 'Groundwater/Recharge_6m.tif'
ds = gdal.Open(str(infn))
gwr = np.array(ds.GetRasterBand(1).ReadAsArray())
gwr[np.isnan(gwr)] = 0
gwr = np.where(gwr <= -9999, 0.00, gwr)

gwr_km3 = gwr * CellArea * 0.000001

gwr_km3_local = np.where(pop2020 > 0 , gwr_km3, 0.00)
sum_basin_gwrkm3 = scipy.ndimage.sum(gwr_km3, basin, basin)
sum_gwrkm3 = scipy.ndimage.sum(gwr_km3, subbas, subbas)

outname = out_dir + 'Groundwater/RechargeVol_6m.tif'
save2File(gwr_km3[:, :], outname, nrows6m, ncols6m, geotr6m)

infn = str(out_dir) + 'Groundwater/RechargeTrend_mm_yr_6m.tif'

ds = gdal.Open(str(infn))
gwTrend = np.array(ds.GetRasterBand(1).ReadAsArray())
gwTrend[np.isnan(gwTrend)] = 0
gwTrend = np.where(gwTrend <= -9999, 0.00, gwTrend)

gwTrend_km3 = gwTrend * CellArea * 0.000001
#sum_basin_gwrkm3 = scipy.ndimage.sum(gwr_km3, basin, basin)

gwTrend_km3_local = np.where(pop2020 > 0 , gwTrend_km3, 0.00)
sum_basin_gwTrendkm3 = scipy.ndimage.sum(gwTrend_km3, basin, basin)
sum_gwTrendkm3 = scipy.ndimage.sum(gwTrend_km3, subbas, subbas)

outname = out_dir + 'Groundwater/RechargeDepthTrendVolkm3_6m.tif'
save2File(gwTrend_km3[:, :], outname, nrows6m, ncols6m, geotr6m)

## Import WBM-TerraClimate Inter- and Intra-Annual Coeff of Variation

In [ ]:
StartYear=2000
EndYear=2020
reso = '6min'

savedCV_dir = str(out_dir) + 'CV-CDF/'

fin = savedCV_dir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYear) + '_Q_DIST_CV_InterAnnual_' + str(reso) + '.tif'
ds = gdal.Open(str(fin))
wbmtc_cv_inter = np.array(ds.GetRasterBand(1).ReadAsArray())
wbmtc_cv_inter[np.isnan(wbmtc_cv_inter)] = 0

fin = savedCV_dir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYear) +'_Q_DIST_meanCV_IntraAnnual_' + str(reso) + '.tif'
ds = gdal.Open(str(fin))
wbmtc_cv_intra = np.array(ds.GetRasterBand(1).ReadAsArray())
wbmtc_cv_intra[np.isnan(wbmtc_cv_intra)] = 0

fin = savedCV_dir + 'WBM_TerraClimate' + str(StartYear) + '-' + str(EndYear) +'_Q_DIST_CV_IntraAnnual_' + str(reso) + '.tif'
ds = gdal.Open(str(fin))
wbmtc_cv_interintra = np.array(ds.GetRasterBand(1).ReadAsArray())
wbmtc_cv_interintra[np.isnan(wbmtc_cv_interintra)] = 0

## Import WBM-TerraClimate Net Irrigation Water Demand LTM and Identify quantiles for headwater cells


In [ ]:
filepath = TCdir + 'WBM_TerraClimate_OUTPUT_IrrNetWD_aLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
irrrovol_mean = dataset.ReadAsArray()

irrromm_mean = irrrovol_mean / CellArea / 0.0000316880878

outname = out_dir + 'OUTPUTQ_LTM/WBM_TerraClimate_OUTPUT_IrrNetWD_mm_aLTM_2000-2020.tif'
save2File(irrromm_mean[:, :], outname, nrows6m, ncols6m, geotr6m)

UpAreaLim = 250
irrrovol_headw = np.where(UpArea > UpAreaLim, irrrovol_mean, 0)
irrromm_headw = np.where(UpArea > UpAreaLim, irrromm_mean, 0)

irrvol_flat = irrrovol_headw.reshape(-1)
df = pd.DataFrame(irrvol_flat, columns = ['IrrNetVol'])
df = df.dropna()
df_flat = df.loc[~(df == 0).any(axis=1)]

irrmm_flat = irrromm_headw.reshape(-1)
df_mm = pd.DataFrame(irrmm_flat, columns = ['IrrNetmm'])
df_mm = df_mm.dropna()
df_mm_flat = df_mm.loc[~(df_mm == 0).any(axis=1)]

## Read in WBM-TerraClimate LTM Raster Discharge Data

Convert to km3/yr

Calculate Available Surface Water volume = Pristine LTM Discharge * 0.2

In [ ]:
print("Read in LTM WBM data..")

# Saved discharge from earlier - newer discharge is unsurprisingly different.

savedQ_dir = str(out_dir) + 'OUTPUTQ_LTM/'

filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_PRIST_aLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMpristQ_ann = dataset.ReadAsArray()

filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_DIST_aLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMdistQ_ann = dataset.ReadAsArray()

filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_PRIST_mLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMpristQ_mo = dataset.ReadAsArray()
    
filepath = savedQ_dir + 'WBM_TerraClimate_OUTPUTQ_DIST_mLTM_2000-2020.tif'
dataset = gdal.Open(filepath)
WBMdistQ_mo = dataset.ReadAsArray()
    
diff = WBMpristQ_mo - WBMdistQ_mo
absdiff = np.absolute(diff)
absfrac = np.where(absdiff[:, :] > 0.0000000, np.where(WBMpristQ_mo == 0.0, 1, absdiff[:, :] / WBMpristQ_mo), 0.0)   

##########################################
#Discharges in km3/yr
##########################################

WBMpristQ_ann_km3yr = WBMpristQ_ann  * 0.031558464
WBMdistQ_ann_km3yr = WBMdistQ_ann  * 0.031558464    
availWBMpristQ_ann_km3yr = WBMpristQ_ann_km3yr * 0.2


# Safe Water Demands and Deficits: local minimum access needs

Safe water demands are defined by four metrics expressed as volume of demand per capita (Table below). Safe water demands are converted to spatially distributed gridded volumes by multiplying the demand metrics by a distributed population dataset for year 2020 (CIESIN 2018) and then summed over river basins. Long term mean annual discharge and available surface water discharge at basin mouth is used to define integrated water flows for the river basins.

| Water Demand Metric | Values (Liters/capita/day)
| --- | --- |
| Domestic Dignity | 50
| Domestic Capability | 100
| All Needs Dignity | 292.85
| All Needs Capability | 405.67

## Raster Safe Water Demands, Recharge, Drawdown, Trends - LOCAL Population

Safe water demands are converted to spatially distributed gridded volumes by multiplying the per capita demand metrics by a distributed population dataset for year 2020 (CIESIN 2018) and then summed over river basins. 

The Safe Water Deficits (sw_deficit_XXXX) are calculated as the amount of water demand not met by the available surface water.

* Safe Water Deficit = Water Demand – Available Surface Water where Water Demand > Available Surface Water

* Available Surface Water = Long Term Mean Pristine Discharge * 0.2

Safe water demand not met by either surface water or groundwater recharge (unmet_sw_deficit_XXXX) is calculated as the amount of safe water deficit in excess of the safely available groundwater recharge:

* Unmet Safe Water Deficit = Safe Water Deficit– Safely Available Groundwater from Recharge where Safe Water Deficit > Safely Available Groundwater from Recharge



In [ ]:
##########################################################################################
# Demands in km3/yr to match Discharge
# Safe Water Deficit in Mill L/yr
# The four water demand metrics for catchment scale analysis (note the different units)
# 1. Domestic (dignity) = 50 L/cap/day
# 2. Domestic (capability) = 100 L/cap/day
# 3. All needs (dignity) = 292.85 L/cap/day
# 4. All needs (capability) = 405.67 L/cap/day
# Convert needs L/d -> 1e-12 km3/L * 365.25 d/y -> 3.6525E-10 km3/yr
# Convert sw_deficit km3/yr -> (1e+12 L/km3) / 1e+6 mill -> 1000000 Mill L/yr
##########################################################################################

target_pop = pop2020

dom_dignity_loc = target_pop * 50 * 3.6525E-10
dom_capability_loc = target_pop * 100 * 3.6525E-10
all_dignity_loc = target_pop * 292.85 * 3.6525E-10
all_capability_loc = target_pop * 405.67 * 3.6525E-10

##########################################
# Convert sw_deficit units to Mill L/yr for comparison to GW
##########################################
sw_deficit_dom_dignity_loc = (np.where(dom_dignity_loc > availWBMpristQ_ann_km3yr, dom_dignity_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_dom_capability_loc = (np.where(dom_capability_loc > availWBMpristQ_ann_km3yr, dom_capability_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_all_dignity_loc = (np.where(all_dignity_loc > availWBMpristQ_ann_km3yr, all_dignity_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_all_capability_loc = (np.where(all_capability_loc > availWBMpristQ_ann_km3yr, all_capability_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000                                  

dom_dignity_loc = dom_dignity_loc * 1000000
dom_capability_loc = dom_capability_loc * 1000000
all_dignity_loc = all_dignity_loc * 1000000
all_capability_loc = all_capability_loc * 1000000

##########################################################################################
# Groundwater Recharge in Mill L/yr
# Multiply GW recharge and drawdown in km3/yr -> (1e+12 L/km3) / 1e+6 -> 1000000 Mill L/yr
##########################################################################################

gw_recharge_loc = (np.where(gwr_km3 > 0, gwr_km3, 0.0)) * 1000000
gw_drawdown_loc = (np.where(gw_km3 < 0, gw_km3, 0.0)) * 1000000

unmet_sw_deficit_dom_dignity_loc = (np.where(sw_deficit_dom_dignity_loc > gw_recharge_loc, sw_deficit_dom_dignity_loc - gw_recharge_loc, 0.0))
unmet_sw_deficit_dom_capability_loc = (np.where(sw_deficit_dom_capability_loc > gw_recharge_loc, sw_deficit_dom_capability_loc - gw_recharge_loc, 0.0))
unmet_sw_deficit_all_dignity_loc = (np.where(sw_deficit_all_dignity_loc > gw_recharge_loc, sw_deficit_all_dignity_loc - gw_recharge_loc, 0.0))
unmet_sw_deficit_all_capability_loc = (np.where(sw_deficit_all_capability_loc > gw_recharge_loc, sw_deficit_all_capability_loc - gw_recharge_loc, 0.0))

##########################################################################################
# Population downstream of areas with unmet SW deficit by All Capability and negative GW trends
# Sum by basin
##########################################################################################
Unmet_sw_deficit_all_capability_GWTrendNeg_loc = np.where(gwTrend_km3 < 0, unmet_sw_deficit_all_capability_loc, 0)
DwnPop_Unmet_sw_deficit_all_capability_GWTrendNeg_loc = np.where(Unmet_sw_deficit_all_capability_GWTrendNeg_loc > 0, target_pop, 0)

outname = out_dir + 'ModelOutput/dom_dignity_loc.tif'
save2File(dom_dignity_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/dom_capability_loc.tif'
save2File(dom_capability_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/all_dignity_loc.tif'
save2File(all_dignity_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/all_capability_loc.tif'
save2File(all_capability_loc[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/sw_deficit_dom_dignity_loc.tif'
save2File(sw_deficit_dom_dignity_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_dom_capability_loc.tif'
save2File(sw_deficit_dom_capability_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_all_dignity_loc.tif'
save2File(sw_deficit_all_dignity_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_all_capability_loc.tif'
save2File(sw_deficit_all_capability_loc[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/gw_recharge_loc.tif'
save2File(gw_recharge_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/gw_drawdown_loc.tif'
save2File(gw_drawdown_loc[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/unmet_sw_deficit_dom_dignity_loc.tif'
save2File(unmet_sw_deficit_dom_dignity_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_dom_capability_loc.tif'
save2File(unmet_sw_deficit_dom_capability_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_all_dignity_loc.tif'
save2File(unmet_sw_deficit_all_dignity_loc[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_all_capability_loc.tif'
save2File(unmet_sw_deficit_all_capability_loc[:, :], outname, nrows6m, ncols6m, geotr6m)


## Raster Safe Water Demands, Recharge, Drawdown, Trends - DOWNSTREAM Population

Safe water demands are converted to spatially distributed gridded volumes by multiplying the per capita demand metrics by a distributed Downstream population dataset for year 2020 (CIESIN 2018) and then summed over river basins. 

The Safe Water Deficits (sw_deficit_XXXX) are calculated as the amount of water demand not met by the available surface water.

* Safe Water Deficit = Water Demand – Available Surface Water where Water Demand > Available Surface Water

* Available Surface Water = Long Term Mean Pristine Discharge * 0.2

Safe water demand not met by either surface water or groundwater recharge (unmet_sw_deficit_XXXX) is calculated as the amount of safe water deficit in excess of the safely available groundwater recharge:

* Unmet Safe Water Deficit = Safe Water Deficit– Safely Available Groundwater from Recharge where Safe Water Deficit > Safely Available Groundwater from Recharge

In [ ]:
##########################################################################################
# Demands in km3/yr to match Discharge
# Safe Water Deficit in Mill L/yr
# The four water demand metrics for catchment scale analysis (note the different units)
# 1. Domestic (dignity) = 50 L/cap/day
# 2. Domestic (capability) = 100 L/cap/day
# 3. All needs (dignity) = 292.85 L/cap/day
# 4. All needs (capability) = 405.67 L/cap/day
# Convert needs L/d -> 1e-12 km3/L * 365.25 d/y -> 3.6525E-10 km3/yr
# Convert sw_deficit km3/yr -> (1e+12 L/km3) / 1e+6 mill -> 1000000 Mill L/yr
##########################################################################################

target_pop = dwnpop2020

dom_dignity_loc = target_pop * 50 * 3.6525E-10
dom_capability_loc = target_pop * 100 * 3.6525E-10
all_dignity_loc = target_pop * 292.85 * 3.6525E-10
all_capability_loc = target_pop * 405.67 * 3.6525E-10

##########################################
# Convert sw_deficit units to Mill L/yr for comparison to GW
##########################################
sw_deficit_dom_dignity_locup = (np.where(dom_dignity_loc > availWBMpristQ_ann_km3yr, dom_dignity_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_dom_capability_locup = (np.where(dom_capability_loc > availWBMpristQ_ann_km3yr, dom_capability_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_all_dignity_locup = (np.where(all_dignity_loc > availWBMpristQ_ann_km3yr, all_dignity_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_all_capability_locup = (np.where(all_capability_loc > availWBMpristQ_ann_km3yr, all_capability_loc - availWBMpristQ_ann_km3yr, 0.0)) * 1000000                                  

dom_dignity_locup = dom_dignity_loc * 1000000
dom_capability_locup = dom_capability_loc * 1000000
all_dignity_locup = all_dignity_loc * 1000000
all_capability_locup = all_capability_loc * 1000000

##########################################################################################
# Groundwater Recharge in Mill L/yr
# Multiply GW recharge and drawdown in km3/yr -> (1e+12 L/km3) / 1e+6 -> 1000000 Mill L/yr
##########################################################################################

unmet_sw_deficit_dom_dignity_locup = (np.where(sw_deficit_dom_dignity_locup > gw_recharge_loc, sw_deficit_dom_dignity_locup - gw_recharge_loc, 0.0))
unmet_sw_deficit_dom_capability_locup = (np.where(sw_deficit_dom_capability_locup > gw_recharge_loc, sw_deficit_dom_capability_locup - gw_recharge_loc, 0.0))
unmet_sw_deficit_all_dignity_locup = (np.where(sw_deficit_all_dignity_locup > gw_recharge_loc, sw_deficit_all_dignity_locup - gw_recharge_loc, 0.0))
unmet_sw_deficit_all_capability_locup = (np.where(sw_deficit_all_capability_locup > gw_recharge_loc, sw_deficit_all_capability_locup - gw_recharge_loc, 0.0))

##########################################################################################
# Population downstream of areas with unmet SW deficity All Capability and negative GW trends
# Sum by basin
##########################################################################################
Unmet_sw_deficit_all_capability_GWTrendNeg_locup = np.where(gwTrend_km3 < 0, unmet_sw_deficit_all_capability_locup, 0)
DwnPop_Unmet_sw_deficit_all_capability_GWTrendNeg_locup = np.where(Unmet_sw_deficit_all_capability_GWTrendNeg_locup > 0, target_pop, 0)

outname = out_dir + 'ModelOutput/dom_dignity_locup.tif'
save2File(dom_dignity_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/dom_capability_locup.tif'
save2File(dom_capability_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/all_dignity_locup.tif'
save2File(all_dignity_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/all_capability_locup.tif'
save2File(all_capability_locup[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/sw_deficit_dom_dignity_locup.tif'
save2File(sw_deficit_dom_dignity_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_dom_capability_locup.tif'
save2File(sw_deficit_dom_capability_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_all_dignity_locup.tif'
save2File(sw_deficit_all_dignity_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_all_capability_locup.tif'
save2File(sw_deficit_all_capability_locup[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/unmet_sw_deficit_dom_dignity_locup.tif'
save2File(unmet_sw_deficit_dom_dignity_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_dom_capability_locup.tif'
save2File(unmet_sw_deficit_dom_capability_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_all_dignity_locup.tif'
save2File(unmet_sw_deficit_all_dignity_locup[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_all_capability_locup.tif'
save2File(unmet_sw_deficit_all_capability_locup[:, :], outname, nrows6m, ncols6m, geotr6m)


## Build E-Flow Rasters based on 10-20-30% change from pristine

Environmental stress is defined as an alteration of flow between pristine (non-human impacted) and disturbed (human impacted) conditions that exceed a given threshold. We derived the pristine and disturbed monthly river flow datasets from the WBM water balance model river discharge outputs (Wisser et al. 2010) at 6-minute grid cell resolution using the TerraClimate high resolution data set of monthly climate forcings (Abatzoglou, et al. 2018) for the period 2000-2020. Long-term mean monthly discharge is calculated for the modelled pristine (non-human impacted) and disturbed (human impacted) discharge from the WBM model over the 2000-2020 time domain to determine the extent of altered flow. 

To quantify the extent of alteration of surface flows, we first calculate the number of months in a year where the contemporary disturbed discharge is more than 10-20-30% different from pre-industrial discharge using the long-term mean gridded discharge data. We then represent this data as the proportion of months in a year with more than 10-20-30% difference (divide by 12 months/year). Finally, we calculate a basin-level estimate of the proportion of grid cells where discharge is more than 10-20-30% different by taking the basin-wide mean of the gridded proportion of months over the actively flowing river extents within the basin. We also calculate the population weighted average of this index using the GPw4 population data (CIESIN 2018) for 2020, and a discharge weighted average of the index using the long-term mean pre-industrial flows from our model runs. River network and basin extents are defined by the WBM water balance model with naming convention taken from the GRDC Major River Basins of the World (GRDC 2020).  

## Basin analysis for ESTRESS, GW Recharge and Trends based on WBM LTM

Safe water demands are summed over river basins. Long term mean annual discharge and available surface water discharge at basin mouth is used to define integrated water flows for the river basins. The Safe Water Deficit by river basin is calculated as the amount of water demand not met by the available surface water.

Safe Water Deficit = Water Demand<sub>Basin</sub> – Available Surface Water<sub>Basin</sub>
where Water Demand<sub>Basin</sub> > Available Surface Water<sub>Basin</sub>

Available Surface Water<sub>Basin</sub> = Long Term Mean Pristine Discharge<sub>Basin</sub> * 0.2

Where we can't meet safe water needs with surface water, we may be able to rely on groundwater resources to meet these needs. Groundwater recharge volumes are summed over river basins for basin-level calculations. We calculate the proportion of groundwater needed to meet the safe water deficits as the ratio of the Safe Water Deficit and the Safely Available Groundwater from Recharge.

Proportion of Groundwater Needed to Meet Safe Water Deficit = 
Safe Water Deficit<sub>Basin</sub> / Safely Available Groundwater from Recharge<sub>Basin</sub>

Finally, safe water demand not met by either surface water or groundwater recharge is calculated as the amount of safe water deficit in excess of the safely available groundwater recharge:

Unmet Safe Water Deficit = Safe Water Deficit<sub>Basin</sub> – Safely Available Groundwater from Recharge<sub>Basin</sub>
where Safe Water Deficit<sub>Basin</sub> > Safely Available Groundwater from Recharge<sub>Basin</sub>

## Link Data to Basin IDs and Output

Basin IDs link to WBM basin shapefiles for map creation


In [ ]:
##########################################
# Demands in km3/yr to match Discharge
# Safe Water Deficit in Mill L/yr
# The four water demand metrics for catchment scale analysis (note the different units)
# 1.	Domestic (dignity) = 50 L/cap/day
# 2.	Domestic (capability) = 100 L/cap/day
# 3.	All needs (dignity) = 292.85 L/cap/day
# 4.	All needs (capability) = 405.67 L/cap/day
# L/d -> 1e-12 km3/L * 365.25 d/y -> 3.6525E-10 km3/yr
# Multiply sw_deficit km3/yr -> (1e+12 L/km3) / 1e+6 -> 1000000 Mill L/yr
##########################################

dom_dignity = sum_basin_pop2020 * 50 * 3.6525E-10
dom_capability = sum_basin_pop2020 * 100 * 3.6525E-10
all_dignity = sum_basin_pop2020 * 292.85 * 3.6525E-10
all_capability = sum_basin_pop2020 * 405.67 * 3.6525E-10

##########################################
# Convert sw_deficit units to Mill L/yr for comparison to GW
# sw_deficit will be sampled at mouth for final table therefor no need to create discharge at mouth aggregate grid
##########################################
sw_deficit_dom_dignity = (np.where(dom_dignity > availWBMpristQ_ann_km3yr, dom_dignity - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_dom_capability = (np.where(dom_capability > availWBMpristQ_ann_km3yr, dom_capability - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_all_dignity = (np.where(all_dignity > availWBMpristQ_ann_km3yr, all_dignity - availWBMpristQ_ann_km3yr, 0.0)) * 1000000
sw_deficit_all_capability = (np.where(all_capability > availWBMpristQ_ann_km3yr, all_capability - availWBMpristQ_ann_km3yr, 0.0)) * 1000000                                  
    
##########################################
# Calculate Deficit proportion of pristine discharges
##########################################

sw_deficit_prop_Prist = (sw_deficit_all_capability / 1000000) / WBMpristQ_ann_km3yr
sw_deficit_prop_AvailPrist = (sw_deficit_all_capability / 1000000) / availWBMpristQ_ann_km3yr

##########################################
# Groundwater Recharge in Mill L/yr
##########################################
gw_recharge = (np.where(sum_basin_gwrkm3 > 0, sum_basin_gwrkm3, 0.0)) * 1000000

gw_drawdown = (np.where(sum_basin_gwkm3 <= 0, sum_basin_gwkm3, 0.0)) * 1000000
gw_drawdown_pos = (np.where(sum_basin_gwkm3 > 0, sum_basin_gwkm3, 0.0)) * 1000000
gw_recharge_depletion = sum_basin_gwkm3 * 1000000

gw_trend = (np.where(sum_basin_gwTrendkm3 < 0, sum_basin_gwTrendkm3, 0.0)) * 1000000

unmet_sw_deficit_dom_dignity = (np.where(sw_deficit_dom_dignity > gw_recharge, sw_deficit_dom_dignity - gw_recharge, 0.0))
unmet_sw_deficit_dom_capability = (np.where(sw_deficit_dom_capability > gw_recharge, sw_deficit_dom_capability - gw_recharge, 0.0))
unmet_sw_deficit_all_dignity = (np.where(sw_deficit_all_dignity > gw_recharge, sw_deficit_all_dignity - gw_recharge, 0.0))
unmet_sw_deficit_all_capability = (np.where(sw_deficit_all_capability > gw_recharge, sw_deficit_all_capability - gw_recharge, 0.0))

##########################################
# Local and Downstream Population under SW Deficity All Capability and Negative GW Trends
# Summed by basin
##########################################
basin_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_locup = scipy.ndimage.sum(DwnPop_Unmet_sw_deficit_all_capability_GWTrendNeg_locup, basin, basin)
basin_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_loc = scipy.ndimage.sum(DwnPop_Unmet_sw_deficit_all_capability_GWTrendNeg_loc, basin, basin)

chglist = [10.0, 20.0, 30.0]
for x in chglist:
    chg = x / 100
    if chg == 0.1:
        chg_dir = 'estress_chg10'
        chg_nm = 'Chg10'
    if chg == 0.2:
        chg_dir = 'estress_chg20'
        chg_nm = 'Chg20'
    if chg == 0.3:
        chg_dir = 'estress_chg30'
        chg_nm = 'Chg30'

    ##########################################
    # ESTRESS = where the difference between Pristine and Disturbed Discharge (absfrac) exceeds 10, 20, or 30% limit
    # absfrac = abs(Pristine_LTM_Q - Disturbed_LTM_Q) / Pristine_LTM_Q 
    # 6-minute raster data is created below then aggregated to basin level in following section 
    ##########################################
    
    print("Create ESTRESS rasters from LTM for " + chg_dir)
    
    diff = WBMpristQ_mo - WBMdistQ_mo
    absdiff = np.absolute(diff)
    absfrac = np.where(absdiff[:, :] > 0.0000000, np.where(WBMpristQ_mo == 0.0, 1, absdiff[:, :] / WBMpristQ_mo), 0.0)   

    estress = np.where(flow_mask == 1.0, 0.0, np.where(absfrac[:, :] >= chg, 1.0, 0.0))  
    estressLTM = np.sum(estress, axis = 0)
    estressLTM = np.where(flow_mask == 0, estressLTM, np.NaN)
    estressLTM = estressLTM[:,:]
    
    UpAreaLim = 250
    #UpAreaLim = 247.3
    #75% = 0.1; 34.02mm
    #50% = 0.01; 3.681mm
    #25% = 0.001; 0.373mm
    #10% = 0.0001; 0.04 mm
    IrrLim = df_mm_flat.IrrNetmm.quantile(0.5)
    
    estressLTM = np.where(flow_mask == 0, np.where(UpArea > UpAreaLim, estressLTM, np.where(irrromm_mean > IrrLim, estressLTM, 0)), np.NaN)

    nans = np.isnan(estressLTM)
    
    estressLTM_CNTnonans = np.where(nans, 0.0, 1.0)
    estressLTM_nonans = np.where(nans, 0.0, estressLTM)

    outname = out_dir + 'ModelOutput/' + chg_dir + '/Estress_WBM_TerraClimate_2000-2020_LTM_' + chg_dir + '.tif'
    save2File(estressLTM[0, :, :], outname, nrows6m, ncols6m, geotr6m)
    
    outname = out_dir + 'ModelOutput/' + chg_dir + '/Estress_WBM_TerraClimate_2000-2020_LTM_CNT' + chg_dir + '.tif'
    save2File(estressLTM_CNTnonans[0, :, :], outname, nrows6m, ncols6m, geotr6m)
    
    outname = out_dir + 'ModelOutput/' + chg_dir + '/Estress_WBM_TerraClimate_2000-2020_LTM_NONANS' + chg_dir + '.tif'
    save2File(estressLTM_nonans[0, :, :], outname, nrows6m, ncols6m, geotr6m)
    
    print("Create ESTRESS Basin/Subbasin Data..")
    
    ###########################
    # Aggregating 6-minute ESTRESS Metric to Basins
    # Arithmentic Mean over Basin
    ###########################
    
    estressLTM_CNTsum = scipy.ndimage.sum(estressLTM_CNTnonans, basin, basin)
    estressLTM_sum = scipy.ndimage.sum(estressLTM_nonans, basin, basin)
    
    estressLTM_basin = (estressLTM_sum / estressLTM_CNTsum) / 12

    outname = out_dir + 'ModelOutput/' + chg_dir + '/Estress_WBM_TerraClimate_2000-2020_LTM_BASIN_' + chg_dir + '.tif'
    save2File(estressLTM_basin[:, :], outname, nrows6m, ncols6m, geotr6m)
    outname = out_dir + 'ModelOutput/' + chg_dir + '/estressLTM_CNTsum_' + chg_dir + '.tif'
    save2File(estressLTM_CNTsum[:, :], outname, nrows6m, ncols6m, geotr6m)   
    outname = out_dir + 'ModelOutput/' + chg_dir + '/estressLTM_sum_' + chg_dir + '.tif'
    save2File(estressLTM_sum[:, :], outname, nrows6m, ncols6m, geotr6m)  
    
    ###########################
    # Aggregating 6-minute ESTRESS Metric to Basins
    # Pop Weighted Mean over Basin
    #################################
        
    estressLTM_NONULL = np.where(flow_mask == 1.0, 0.0, np.where(estressLTM >= 0, estressLTM, 0.0))
    tempcal = estressLTM_NONULL * pop2020
    basin_popwgtESTRESS = (scipy.ndimage.sum(tempcal, basin, basin) / sum_basin_pop2020 ) / 12
    
    cnt = np.where(flow_mask == 1.0, 0.0, np.where(estressLTM >= 0, 1, 0))
    basin_meanESTRESS = (scipy.ndimage.sum(estressLTM_NONULL, basin, basin) / scipy.ndimage.sum(cnt, basin, basin)) / 12
    outname = out_dir + 'ModelOutput/' + chg_dir + '/basin_meanESTRESS' + chg_dir + '.tif'
    save2File(basin_meanESTRESS[:, :], outname, nrows6m, ncols6m, geotr6m)
         
    ###########################
    # Aggregating 6-minute ESTRESS Metric to Basins
    # Discharge Weighted Mean over Basin 
    #################################
        
    tempcal = estressLTM_NONULL * WBMpristQ_ann_km3yr
    sum_basin_Q = scipy.ndimage.sum(WBMpristQ_ann_km3yr, basin, basin)
    basin_QwgtESTRESS = (scipy.ndimage.sum(tempcal, basin, basin) / sum_basin_Q ) / 12

    ###########################
    # Aggregating 6-minute Coeff of Variation of ESTRESS Metric to Basins
    # Pop Weighted Mean of CV over Basin 
    #################################

    tempcal = np.where(flow_mask == 1.0, 0.0, wbmtc_cv_inter) * pop2020
    basin_popwgtCVInter = (scipy.ndimage.sum(tempcal, basin, basin) / sum_basin_pop2020 )

    tempcal = np.where(flow_mask == 1.0, 0.0, wbmtc_cv_intra) * pop2020
    basin_popwgtCVIntra = (scipy.ndimage.sum(tempcal, basin, basin) / sum_basin_pop2020 )
    
    tempcal = np.where(flow_mask == 1.0, 0.0, wbmtc_cv_interintra) * pop2020
    basin_popwgtCVInterIntra = (scipy.ndimage.sum(tempcal, basin, basin) / sum_basin_pop2020 )

    #################################
    #Build Basin/Subbasin Tables
    #################################
    
    shp_filename = out_dir + 'RiverBasin/HydroSTN30_Confluence_6m.shp'
    ds=ogr.Open(shp_filename)
    lyr=ds.GetLayer()

    li_values = list()
    liincr_values = list()
    lidecr_values = list()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('ID')
        mx, my = geom.GetX(), geom.GetY()
        px = floor((180 + mx) / 0.1)
        py = floor((90 - my) / 0.1)
        li_values.append([feat_id, estressLTM[0, py, px]])

    
    shp_filename = out_dir + 'RiverBasin/HydroSTN06_BasinMouth.shp'
    ds=ogr.Open(shp_filename)
    lyr=ds.GetLayer()

    values1 = list()
    values2 = list()
    values3 = list()
    values4 = list()
    values5 = list()
    values6 = list()
    values7 = list()
    values8 = list()
    values9 = list()
    values10 = list()
    values11 = list()
    values12 = list()
    values13 = list()
    values14 = list()
    values15 = list()
    values16 = list()
    values17 = list()
    values18 = list()
    values19 = list()
    values20 = list()
    values21 = list()
    values22 = list()
    values23 = list()
    values24 = list()
    values25 = list()
    values26 = list()
    
    for feat in lyr:
        geom = feat.GetGeometryRef()
        feat_id = feat.GetField('ID')
        mx, my = geom.GetX(), geom.GetY()
        px = floor((180 + mx) / 0.1)
        py = floor((90 - my) / 0.1)
        values1.append([feat_id, sum_basin_area[py, px]])
        values2.append([feat_id, WBMpristQ_ann_km3yr[py, px]])
        values3.append([feat_id, availWBMpristQ_ann_km3yr[py, px]])
        values4.append([feat_id, WBMdistQ_ann_km3yr[py, px]])
        values5.append([feat_id, sum_basin_pop2020[py, px]])
        values6.append([feat_id, estressLTM_basin[py, px]])
        values7.append([feat_id, basin_popwgtESTRESS[py, px]])
        values8.append([feat_id, basin_QwgtESTRESS[py, px]])
        values9.append([feat_id, dom_dignity[py, px]])
        values10.append([feat_id, dom_capability[py, px]])
        values11.append([feat_id, sw_deficit_dom_dignity[py, px]])
        values12.append([feat_id, sw_deficit_dom_capability[py, px]])
        values13.append([feat_id, sw_deficit_all_dignity[py, px]])
        values14.append([feat_id, sw_deficit_all_capability[py, px]]) 
        values15.append([feat_id, gw_recharge[py, px]]) 
        values16.append([feat_id, gw_drawdown[py, px]]) 
        values17.append([feat_id, gw_trend[py, px]]) 
        values18.append([feat_id, unmet_sw_deficit_dom_dignity[py, px]])
        values19.append([feat_id, unmet_sw_deficit_dom_capability[py, px]])
        values20.append([feat_id, unmet_sw_deficit_all_dignity[py, px]])
        values21.append([feat_id, unmet_sw_deficit_all_capability[py, px]])
        values22.append([feat_id, basin_popwgtCVInter[py, px]])
        values23.append([feat_id, basin_popwgtCVIntra[py, px]])
        values24.append([feat_id, basin_popwgtCVInterIntra[py, px]])   
        values25.append([feat_id, basin_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_locup[py, px]])   
        values26.append([feat_id, gw_recharge_depletion[py, px]]) 
        
    df1 = pd.DataFrame(values1)
    df2 = pd.DataFrame(values2)
    df3 = pd.DataFrame(values3)
    df4 = pd.DataFrame(values4)
    df5 = pd.DataFrame(values5)
    df6 = pd.DataFrame(values6)
    df7 = pd.DataFrame(values7)
    df8 = pd.DataFrame(values8)
    df9 = pd.DataFrame(values9)
    df10 = pd.DataFrame(values10)
    df11 = pd.DataFrame(values11)
    df12 = pd.DataFrame(values12)
    df13 = pd.DataFrame(values13)
    df14 = pd.DataFrame(values14)
    df15 = pd.DataFrame(values15)
    df16 = pd.DataFrame(values16)
    df17 = pd.DataFrame(values17)
    df18 = pd.DataFrame(values18)
    df19 = pd.DataFrame(values19)
    df20 = pd.DataFrame(values20)
    df21 = pd.DataFrame(values21)
    df22 = pd.DataFrame(values22)
    df23 = pd.DataFrame(values23)
    df24 = pd.DataFrame(values24)
    df25 = pd.DataFrame(values25)
    df26 = pd.DataFrame(values26)
    
    df1.columns =['ID', 'sum_basin_area']
    df2.columns =['ID', 'WBMpristQ_ann_km3yr']
    df3.columns =['ID', 'availWBMpristQ_ann_km3yr']
    df4.columns =['ID', 'WBMdistQ_ann_km3yr']
    df5.columns =['ID', 'sum_basin_pop2020']
    df6.columns =['ID', 'estressLTM_basin']
    df7.columns =['ID', 'basin_popwgtESTRESS']
    df8.columns =['ID', 'basin_QwgtESTRESS']
    df9.columns =['ID', 'dom_dignity']
    df10.columns =['ID', 'dom_capability']
    df11.columns =['ID', 'sw_deficit_dom_dignity']
    df12.columns =['ID', 'sw_deficit_dom_capability']
    df13.columns =['ID', 'sw_deficit_all_dignity']
    df14.columns =['ID', 'sw_deficit_all_capability']
    df15.columns =['ID', 'gw_recharge']
    df16.columns =['ID', 'gw_drawdown']
    df17.columns =['ID', 'gw_trend']
    df18.columns =['ID', 'unmet_sw_deficit_dom_dignity']
    df19.columns =['ID', 'unmet_sw_deficit_dom_capability']
    df20.columns =['ID', 'unmet_sw_deficit_all_dignity']
    df21.columns =['ID', 'unmet_sw_deficit_all_capability']
    df22.columns =['ID', 'basin_popwgtCVInter']
    df23.columns =['ID', 'basin_popwgtCVIntra']
    df24.columns =['ID', 'basin_popwgtCVInterIntra']
    df25.columns =['ID', 'basin_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_locup']
    df26.columns =['ID', 'gw_recharge_depletion']
    
    
    df = pd_basin_name
    df = pd.merge(df, df1, on=['ID'])
    df = pd.merge(df, df2, on=['ID'])
    df = pd.merge(df, df3, on=['ID'])
    df = pd.merge(df, df4, on=['ID'])
    df = pd.merge(df, df5, on=['ID'])
    df = pd.merge(df, df6, on=['ID'])
    df = pd.merge(df, df7, on=['ID'])
    df = pd.merge(df, df8, on=['ID'])
    df = pd.merge(df, df9, on=['ID'])
    df = pd.merge(df, df10, on=['ID'])
    df = pd.merge(df, df11, on=['ID'])
    df = pd.merge(df, df12, on=['ID'])
    df = pd.merge(df, df13, on=['ID'])
    df = pd.merge(df, df14, on=['ID'])
    df = pd.merge(df, df15, on=['ID'])
    df = pd.merge(df, df16, on=['ID'])
    df = pd.merge(df, df17, on=['ID'])
    df = pd.merge(df, df18, on=['ID'])
    df = pd.merge(df, df19, on=['ID'])
    df = pd.merge(df, df20, on=['ID'])
    df = pd.merge(df, df21, on=['ID'])
    df = pd.merge(df, df22, on=['ID'])
    df = pd.merge(df, df23, on=['ID'])
    df = pd.merge(df, df24, on=['ID'])
    df = pd.merge(df, df25, on=['ID'])
    df = pd.merge(df, df26, on=['ID'])
    
    #df = pd.DataFrame(li_values)  
        
    filenmXL = out_dir + 'ModelOutput/Tables/BASIN_Stats_ESTRESS_GWr' + chg_nm + '.xlsx'
    with pd.ExcelWriter(filenmXL) as writer:
        df.to_excel(writer, index=False, sheet_name='Basin_Frac_ALL', header=['ID', 'Basin Name', 'Basin Area (km2)', 
                                                                              'Pristine Discharge (km3/yr)', 'Available Pristine Discharge (km3/yr)', 'Contemporary Discharge (km3/yr)', 
                                                                              'Total Population 2020', 'Proportion of Months where Flows Differ (Basin mean)', 'Proportion of Months where Flows Differ (Population weighted)', 'Proportion of Months where Flows Differ (Discharge weighted)', 
                                                                              'Domestic Dignity Need (km3/yr)', 'Domestic Capability Need (km3/yr)', 'Domestic Dignity Safe Water Deficit (mill L/yr)', 'Domestic Capability Safe Water Deficit (mill L/yr)', 'All Dignity Safe Water Deficit (mill L/yr)', 'All Capability Safe Water Deficit (mill L/yr)', 
                                                                              'Safely Available GW RECHARGE mill L/yr', 'Net GW Depletion mill L/yr', 'Net GW Trend mill L/yr', 'UNMET Domestic Dignity Safe Water Deficit (mill L/yr)', 'UNMET Domestic Capability Safe Water Deficit (mill L/yr)', 'UNMET All Dignity Safe Water Deficit (mill L/yr)', 'UNMET All Capability Safe Water Deficit (mill L/yr)', 
                                                                              'CV Inter-Annual Discharge', 'CV Intra-Annual Discharge', 'CV Inter-Intra Discharge Combo', 'Downstream Population with Unmet_sw_deficit_all_capability and Negative GWTrends', 'Recharge rate - depletion mill L/yr'])
    del df1
    del df2
    del df3
    del df4
    del df5
    del df6
    del df7
    del df8
    del df9
    del df10
    del df11
    del df12
    del df13
    del df14
    del df15
    del df16
    del df17
    del df18
    del df19
    del df20
    del df21
    del df22
    del df23
    del df24
    del df25
    del df
    

## Subbasin analysis for ESTRESS, GW Recharge and Trends based on WBM LTM

Same analysis from above cell for Basins is applied here

In [ ]:
##########################################
#Discharges in km3/yr by subbas
##########################################
WBMpristQ_ann_subbas = scipy.ndimage.maximum(WBMpristQ_ann, subbas, subbas)
WBMdistQ_ann_subbas = scipy.ndimage.maximum(WBMdistQ_ann, subbas, subbas)
WBMpristQ_ann_km3yr_subbas = WBMpristQ_ann_subbas  * 0.031558464
WBMdistQ_ann_km3yr_subbas = WBMdistQ_ann_subbas  * 0.031558464
availWBMpristQ_ann_km3yr_subbas = WBMpristQ_ann_km3yr_subbas * 0.2

print("Build Avail water, Needs, GW recharge/drawdown..")
##########################################
# Demands in km3/yr to match Discharge
# Safe Water Deficit in Mill L/yr
# The four water demand metrics for catchment scale analysis (note the different units)
# 1.	Domestic (dignity) = 50 L/cap/day
# 2.	Domestic (capability) = 100 L/cap/day
# 3.	All needs (dignity) = 292.85 L/cap/day
# 4.	All needs (capability) = 405.67 L/cap/day
# L/d -> 1e-12 km3/L * 365.25 d/y -> 3.6525E-10 km3/yr
# Multiply sw_deficit km3/yr -> (1e+12 L/km3) / 1e+6 -> 1000000 Mill L/yr
##########################################

dom_dignity_subbas = sum_pop2020 * 50 * 3.6525E-10
dom_capability_subbas = sum_pop2020 * 100 * 3.6525E-10
all_dignity_subbas = sum_pop2020 * 292.85 * 3.6525E-10
all_capability_subbas = sum_pop2020 * 405.67 * 3.6525E-10

##########################################
# Convert sw_deficit units to Mill L/yr for comparison to GW
##########################################
sw_deficit_dom_dignity_subbas = (np.where(dom_dignity_subbas > availWBMpristQ_ann_km3yr_subbas, dom_dignity_subbas - availWBMpristQ_ann_km3yr_subbas, 0.0)) * 1000000
sw_deficit_dom_capability_subbas = (np.where(dom_capability_subbas > availWBMpristQ_ann_km3yr_subbas, dom_capability_subbas - availWBMpristQ_ann_km3yr_subbas, 0.0)) * 1000000
sw_deficit_all_dignity_subbas = (np.where(all_dignity_subbas > availWBMpristQ_ann_km3yr_subbas, all_dignity_subbas - availWBMpristQ_ann_km3yr_subbas, 0.0)) * 1000000
sw_deficit_all_capability_subbas = (np.where(all_capability_subbas > availWBMpristQ_ann_km3yr_subbas, all_capability_subbas - availWBMpristQ_ann_km3yr_subbas, 0.0)) * 1000000                                  

##########################################
# Groundwater Recharge in Mill L/yr
##########################################
gw_recharge_subbas = (np.where(sum_gwrkm3 > 0, sum_gwrkm3, 0.0)) * 1000000
gw_drawdown_subbas = (np.where(sum_gwkm3 < 0, sum_gwkm3, 0.0)) * 1000000

unmet_sw_deficit_dom_dignity_subbas = (np.where(sw_deficit_dom_dignity_subbas > gw_recharge_subbas, sw_deficit_dom_dignity_subbas - gw_recharge_subbas, 0.0))
unmet_sw_deficit_dom_capability_subbas = (np.where(sw_deficit_dom_capability_subbas > gw_recharge_subbas, sw_deficit_dom_capability_subbas - gw_recharge_subbas, 0.0))
unmet_sw_deficit_all_dignity_subbas = (np.where(sw_deficit_all_dignity_subbas > gw_recharge_subbas, sw_deficit_all_dignity_subbas - gw_recharge_subbas, 0.0))
unmet_sw_deficit_all_capability_subbas = (np.where(sw_deficit_all_capability_subbas > gw_recharge_subbas, sw_deficit_all_capability_subbas - gw_recharge_subbas, 0.0))

##########################################
# Local and Downstream Population under SW Deficity All Capability and Negative GW Trends
# Summed by subbasi
##########################################
subbas_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_locup = scipy.ndimage.sum(DwnPop_Unmet_sw_deficit_all_capability_GWTrendNeg_locup, subbas, subbas)
subbas_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_loc = scipy.ndimage.sum(DwnPop_Unmet_sw_deficit_all_capability_GWTrendNeg_loc, subbas, subbas)

outname = out_dir + 'ModelOutput/dom_dignity_subbas.tif'
save2File(dom_dignity_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/dom_capability_subbas.tif'
save2File(dom_capability_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/all_dignity_subbas.tif'
save2File(all_dignity_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/all_capability_subbas.tif'
save2File(all_capability_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/sw_deficit_dom_dignity_subbas.tif'
save2File(sw_deficit_dom_dignity_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_dom_capability_subbas.tif'
save2File(sw_deficit_dom_capability_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_all_dignity_subbas.tif'
save2File(sw_deficit_all_dignity_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/sw_deficit_all_capability_subbas.tif'
save2File(sw_deficit_all_capability_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/gw_recharge_subbas.tif'
save2File(gw_recharge_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/gw_drawdown_subbas.tif'
save2File(gw_drawdown_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)

outname = out_dir + 'ModelOutput/unmet_sw_deficit_dom_dignity_subbas.tif'
save2File(unmet_sw_deficit_dom_dignity_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_dom_capability_subbas.tif'
save2File(unmet_sw_deficit_dom_capability_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_all_dignity_subbas.tif'
save2File(unmet_sw_deficit_all_dignity_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)
outname = out_dir + 'ModelOutput/unmet_sw_deficit_all_capability_subbas.tif'
save2File(unmet_sw_deficit_all_capability_subbas[:, :], outname, nrows6m, ncols6m, geotr6m)


#################################
#Build Basin/Subbasin Tables
#################################

shp_filename = out_dir + 'RiverBasin/HydroSTN30_Confluence_6m.shp'
ds=ogr.Open(shp_filename)
lyr=ds.GetLayer()

shp_filename = out_dir + 'RiverBasin/HydroSTN30_Confluence_6m.shp'
ds=ogr.Open(shp_filename)
lyr=ds.GetLayer()

print(lyr)
values1 = list()
values2 = list()
values3 = list()
values4 = list()
values5 = list()
values6 = list()
values7 = list()
values8 = list()
values9 = list()
values10 = list()
values11 = list()
values12 = list()
values13 = list()
values14 = list()
values15 = list()
values16 = list()
values17 = list()
values18 = list()

for feat in lyr:
    geom = feat.GetGeometryRef()
    feat_id = feat.GetField('ID')
    mx, my = geom.GetX(), geom.GetY()
    px = floor((180 + mx) / 0.1)
    py = floor((90 - my) / 0.1)
    values1.append([feat_id, sum_area[py, px]])
    values2.append([feat_id, WBMpristQ_ann_km3yr[py, px]])
    values3.append([feat_id, availWBMpristQ_ann_km3yr[py, px]])
    values4.append([feat_id, WBMdistQ_ann_km3yr[py, px]])
    values5.append([feat_id, sum_pop2020[py, px]])
    values6.append([feat_id, dom_dignity_subbas[py, px]])
    values7.append([feat_id, dom_capability_subbas[py, px]])
    values8.append([feat_id, sw_deficit_dom_dignity_subbas[py, px]])
    values9.append([feat_id, sw_deficit_dom_capability_subbas[py, px]])
    values10.append([feat_id, sw_deficit_all_dignity_subbas[py, px]])
    values11.append([feat_id, sw_deficit_all_capability_subbas[py, px]]) 
    values12.append([feat_id, gw_recharge_subbas[py, px]]) 
    values13.append([feat_id, gw_drawdown_subbas[py, px]]) 
    values14.append([feat_id, unmet_sw_deficit_dom_dignity_subbas[py, px]])
    values15.append([feat_id, unmet_sw_deficit_dom_capability_subbas[py, px]])
    values16.append([feat_id, unmet_sw_deficit_all_dignity_subbas[py, px]])
    values17.append([feat_id, unmet_sw_deficit_all_capability_subbas[py, px]])
    values18.append([feat_id, subbas_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_locup[py, px]])

df1 = pd.DataFrame(values1)
df2 = pd.DataFrame(values2)
df3 = pd.DataFrame(values3)
df4 = pd.DataFrame(values4)
df5 = pd.DataFrame(values5)
df6 = pd.DataFrame(values6)
df7 = pd.DataFrame(values7)
df8 = pd.DataFrame(values8)
df9 = pd.DataFrame(values9)
df10 = pd.DataFrame(values10)
df11 = pd.DataFrame(values11)
df12 = pd.DataFrame(values12)
df13 = pd.DataFrame(values13)
df14 = pd.DataFrame(values14)
df15 = pd.DataFrame(values15)
df16 = pd.DataFrame(values16)
df17 = pd.DataFrame(values17)
df18 = pd.DataFrame(values17)

df1.columns =['ID', 'sum_area']
df2.columns =['ID', 'WBMpristQ_ann_km3yr']
df3.columns =['ID', 'availWBMpristQ_ann_km3yr']
df4.columns =['ID', 'WBMdistQ_ann_km3yr']
df5.columns =['ID', 'sum_pop2020']
df6.columns =['ID', 'dom_dignity']
df7.columns =['ID', 'dom_capability']
df8.columns =['ID', 'sw_deficit_dom_dignity']
df9.columns =['ID', 'sw_deficit_dom_capability']
df10.columns =['ID', 'sw_deficit_all_dignity']
df11.columns =['ID', 'sw_deficit_all_capability']
df12.columns =['ID', 'gw_recharge']
df13.columns =['ID', 'gw_drawdown']
df14.columns =['ID', 'unmet_sw_deficit_dom_dignity']
df15.columns =['ID', 'unmet_sw_deficit_dom_capability']
df16.columns =['ID', 'unmet_sw_deficit_all_dignity']
df17.columns =['ID', 'unmet_sw_deficit_all_capability']
df18.columns =['ID', 'subbas_dwnpop2020_Unmet_sw_deficit_all_capability_GWTrendNeg_locup']

df = pd_basin_name
df = pd.merge(df, df1, on=['ID'])
df = pd.merge(df, df2, on=['ID'])
df = pd.merge(df, df3, on=['ID'])
df = pd.merge(df, df4, on=['ID'])
df = pd.merge(df, df5, on=['ID'])
df = pd.merge(df, df6, on=['ID'])
df = pd.merge(df, df7, on=['ID'])
df = pd.merge(df, df8, on=['ID'])
df = pd.merge(df, df9, on=['ID'])
df = pd.merge(df, df10, on=['ID'])
df = pd.merge(df, df11, on=['ID'])
df = pd.merge(df, df12, on=['ID'])
df = pd.merge(df, df13, on=['ID'])
df = pd.merge(df, df14, on=['ID'])
df = pd.merge(df, df15, on=['ID'])
df = pd.merge(df, df16, on=['ID'])
df = pd.merge(df, df17, on=['ID'])
df = pd.merge(df, df18, on=['ID'])

filenmXL = out_dir + 'ModelOutput/Tables/SUBBASIN_Stats_GWr.xlsx'
with pd.ExcelWriter(filenmXL) as writer:
    df.to_excel(writer, index=False, sheet_name='Basin_Frac_ALL', header=['ID', 'Basin Name', 'SubBasin Area (km2)', 'Pristine Discharge (km3/yr)', 'Available Pristine Discharge (km3/yr)', 'Contemporary Discharge (km3/yr)', 'Total Population 2020', 'Domestic Dignity Need (km3/yr)', 'Domestic Capability Need (km3/yr)', 'Domestic Dignity Safe Water Deficit (mill L/yr)', 'Domestic Capability Safe Water Deficit (mill L/yr)', 'All Dignity Safe Water Deficit (mill L/yr)', 'All Capability Safe Water Deficit (mill L/yr)', 'Safely Available GW RECHARGE mill L/yr', 'Net GW Depletion mill L/yr', 'UNMET Domestic Dignity Safe Water Deficit (mill L/yr)', 'UNMET Domestic Capability Safe Water Deficit (mill L/yr)', 'UNMET All Dignity Safe Water Deficit (mill L/yr)', 'UNMET All Capability Safe Water Deficit (mill L/yr)', 'Downstream Population with Unmet_sw_deficit_all_capability and Negative GWTrends'])
del df1
del df2
del df3
del df4
del df5
del df6
del df7
del df8
del df9
del df10
del df11
del df12
del df13
del df14
del df15
del df16
del df17
del df18
del df
    